In [16]:
!pip install ipykernel 
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd

In [18]:
with pd.ExcelFile('Inputs.xlsx') as book:
    risk_ranking_weights = pd.read_excel(book, 'Risk Ranking Weights')
    non_union_rates = pd.read_excel(book, 'Non-Union Rates')
    union_rates = pd.read_excel(book, 'Union Rates')
    checkpoint_costs = pd.read_excel(book, 'Checkpoint Costs')
    project_info = pd.read_excel(book, 'Project Info')
    df = pd.read_excel(book, 'merged_data_union')

In [19]:
# Fix date formating
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format = 'mixed')
df['DateFixed'] = pd.to_datetime(df['DateFixed'], format = 'mixed')

In [20]:
df['ProjectName'] = df['Project'].str.rsplit(n=1).str[0]
df['ProjectNumber'] = df['Project'].str.rsplit(n=1).str[1].str.lstrip('(').str.rstrip(')')
df = df.drop(columns='Project')
df

,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,Checkpoint ID,Checkpoint Item,Daily/Weekly,AM/PM,Notes,ResponsibleParty,Location,Union / Non-Union,ProjectName,ProjectNumber
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,HN-2002-2004-6,"All coolers, inside and out are clean and stoc...",WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,HN-2002-2004-6,"All coolers, inside and out are clean and stoc...",WEEKLY,NaN,"Auditor to select|deficiency, Reason Code and ...",JA,Western (49-1),Union,Suncor - Mount Logan,49
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,HN-2002-2004-6,"All coolers, inside and out are clean and stoc...",WEEKLY,NaN,| Fridge 1 not cooling,JA,Western (49-1),Union,Suncor - Mount Logan,49
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,HN-2002-2004-6,"All coolers, inside and out are clean and stoc...",WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,HN-2002-2004-7,Till /POS and counter area is clean and organi...,WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22050,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,SLL-2023-1002-6,Breakfast Bar; must include yogurt,DAILY,NaN,| Nights forgot to take a picture,CUL,Eastern (21-1),Non-Union,Snow Lake,21
22051,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,SLL-2023-1002-7,A daily Breakfast Special is available,DAILY,NaN,Quality Auditing Compliance-Photo required or ...,CUL,NaN,Non-Union,Snow Lake,21
22052,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,SLL-2023-1002-7,A daily Breakfast Special is available,DAILY,NaN,NaN,CUL,NaN,Non-Union,Snow Lake,21
22053,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,SLL-2023-1002-7,A daily Breakfast Special is available,DAILY,NaN,Quality Auditing|Compliance-Photo required-LG,CUL,Eastern (21-1),Non-Union,Snow Lake,21


In [21]:
# Step 0: Strip spaces from all column names just in case
df.columns = df.columns.str.strip()
checkpoint_costs.columns = checkpoint_costs.columns.str.strip()
project_info.columns = project_info.columns.str.strip()
union_rates.columns = union_rates.columns.str.strip()
non_union_rates.columns = non_union_rates.columns.str.strip()
risk_ranking_weights.columns = risk_ranking_weights.columns.str.strip()

# Step 0.1: Replace empty material cost cells with zero
checkpoint_costs = checkpoint_costs.fillna(0)

# Step 0.2: Adjust weight by adding 1 to the percentage increase
risk_ranking_weights['Weights'] = risk_ranking_weights['Weights'] + 1

In [22]:
# Step 1: Merge checkpoint data (time + material)
df = df.merge(
    checkpoint_costs[['Checkpoint ID', 'Time taken to complete task (in mins)', 'Material Cost']],
    on='Checkpoint ID',
    how='left')
    #on=['Checkpoint ID', 'Risk Ranking', 'Responsible Party', 'Checklist ID', 'Checklist Category', 'Checklist Code', 'Checklist Item', 'Checkpoint ID', 'Checkpoint Item'],
df

,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,Daily/Weekly,AM/PM,Notes,ResponsibleParty,Location,Union / Non-Union,ProjectName,ProjectNumber,Time taken to complete task (in mins),Material Cost
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,WEEKLY,NaN,"Auditor to select|deficiency, Reason Code and ...",JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,WEEKLY,NaN,| Fridge 1 not cooling,JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,WEEKLY,NaN,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24111,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,DAILY,NaN,| Nights forgot to take a picture,CUL,Eastern (21-1),Non-Union,Snow Lake,21,3,0.0
24112,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,DAILY,NaN,Quality Auditing Compliance-Photo required or ...,CUL,NaN,Non-Union,Snow Lake,21,10,0.0
24113,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,DAILY,NaN,NaN,CUL,NaN,Non-Union,Snow Lake,21,10,0.0
24114,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,DAILY,NaN,Quality Auditing|Compliance-Photo required-LG,CUL,Eastern (21-1),Non-Union,Snow Lake,21,10,0.0


In [23]:
project_info = project_info.rename(columns={'Project': 'ProjectName', 'Labour type': 'LabourType'})

df = df.merge(
    project_info[['ProjectName', 'LabourType', 'Max Occupancy']],
    on = 'ProjectName',
    how='left'
)
df

,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,Notes,ResponsibleParty,Location,Union / Non-Union,ProjectName,ProjectNumber,Time taken to complete task (in mins),Material Cost,LabourType,Max Occupancy
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0,Union,1500
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,"Auditor to select|deficiency, Reason Code and ...",JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0,Union,1243
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,| Fridge 1 not cooling,JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0,Union,1243
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0,Union,1500
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Western (48-1),Union,Suncor - Firebag,48,15,0.0,Union,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24111,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,| Nights forgot to take a picture,CUL,Eastern (21-1),Non-Union,Snow Lake,21,3,0.0,Non-Union,445
24112,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Quality Auditing Compliance-Photo required or ...,CUL,NaN,Non-Union,Snow Lake,21,10,0.0,Non-Union,445
24113,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,CUL,NaN,Non-Union,Snow Lake,21,10,0.0,Non-Union,445
24114,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Quality Auditing|Compliance-Photo required-LG,CUL,Eastern (21-1),Non-Union,Snow Lake,21,10,0.0,Non-Union,445


In [24]:
# Step 2: Add union status and combine rate tables
non_union_rates['LabourType'] = 'Non-Union'
union_rates['LabourType'] = 'Union'

combined_rates = pd.concat([non_union_rates, union_rates], ignore_index=True)

combined_rates['Minute Rate'] = combined_rates['Hourly Rate'] / 60

# Step 2.5: Rename to standard column name for ease
combined_rates = combined_rates.rename(columns={'Minute Rate': 'MinuteRate', 'Code': 'ResponsibleParty'})

In [25]:
# Step 3: Merge rate based on Responsible Party and Union / Non-Union
df = df.merge(
    combined_rates[['ResponsibleParty', 'MinuteRate', 'LabourType']],
    on=['ResponsibleParty', 'LabourType'],
    how='left'
)
df


,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,ResponsibleParty,Location,Union / Non-Union,ProjectName,ProjectNumber,Time taken to complete task (in mins),Material Cost,LabourType,Max Occupancy,MinuteRate
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0,Union,1500,0.407333
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0,Union,1243,0.407333
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,JA,Western (49-1),Union,Suncor - Mount Logan,49,10,0.0,Union,1243,0.407333
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,JA,Western (48-1),Union,Suncor - Firebag,48,10,0.0,Union,1500,0.407333
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,JA,Western (48-1),Union,Suncor - Firebag,48,15,0.0,Union,1500,0.407333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24111,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,CUL,Eastern (21-1),Non-Union,Snow Lake,21,3,0.0,Non-Union,445,0.316667
24112,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,CUL,NaN,Non-Union,Snow Lake,21,10,0.0,Non-Union,445,0.316667
24113,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,CUL,NaN,Non-Union,Snow Lake,21,10,0.0,Non-Union,445,0.316667
24114,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,CUL,Eastern (21-1),Non-Union,Snow Lake,21,10,0.0,Non-Union,445,0.316667


In [26]:
# Step 4: Merge risk ranking weights
risk_ranking_weights = risk_ranking_weights.rename(columns={'Risk Ranking': 'RiskRanking', 'Weight': 'RiskWeight'})
df = df.merge(risk_ranking_weights, on='RiskRanking', how='left')

In [27]:
# Step 5: Calculate re-do cost
################### DOESNT INCLUDE RISK RANKINGS YET BC IM CROSS CHECKING WITH NATHANS POWERBI
df['Cost to re-do (based on hourly wage and Risk Ranking Weights)'] = (
    df['Time taken to complete task (in mins)'] *
    df['MinuteRate'] 
)

In [28]:
# Step 6: Total cost
df['Total cost to re-do task'] = (
    df['Cost to re-do (based on hourly wage and Risk Ranking Weights)'] +
    df['Material Cost']
)
df

,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,ProjectName,ProjectNumber,Time taken to complete task (in mins),Material Cost,LabourType,Max Occupancy,MinuteRate,Weights,Cost to re-do (based on hourly wage and Risk Ranking Weights),Total cost to re-do task
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Suncor - Firebag,48,10,0.0,Union,1500,0.407333,1.0,4.073333,4.073333
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Suncor - Mount Logan,49,10,0.0,Union,1243,0.407333,1.0,4.073333,4.073333
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Suncor - Mount Logan,49,10,0.0,Union,1243,0.407333,1.0,4.073333,4.073333
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Suncor - Firebag,48,10,0.0,Union,1500,0.407333,1.0,4.073333,4.073333
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Suncor - Firebag,48,15,0.0,Union,1500,0.407333,1.0,6.110000,6.110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24111,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Snow Lake,21,3,0.0,Non-Union,445,0.316667,1.1,0.950000,0.950000
24112,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Snow Lake,21,10,0.0,Non-Union,445,0.316667,1.0,3.166667,3.166667
24113,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Snow Lake,21,10,0.0,Non-Union,445,0.316667,1.0,3.166667,3.166667
24114,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Snow Lake,21,10,0.0,Non-Union,445,0.316667,1.0,3.166667,3.166667


In [29]:
df['Cost to re-do (based on hourly wage and Risk Ranking Weights)'].sum(0)

np.float64(80001.72283333333)

In [30]:
df.to_excel('Outputs.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'Outputs.xlsx'